In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/photobooth-images/banner.jpg
/kaggle/input/photobooth-images/home.jpg


In [2]:
!pip install streamlit streamlit-webrtc pyngrok diffusers transformers accelerate torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.8/216.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 14.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
%%writefile app.py
import streamlit as st
from PIL import Image, ImageDraw, ImageFont
import io
import os
from datetime import datetime
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline

# --- Init Page State ---
if "page" not in st.session_state:
    st.session_state.page = "home"
if "photos" not in st.session_state:
    st.session_state.photos = []

# --- Home Page ---
if st.session_state.page == "home":
    # --- Global Styling ---
    st.markdown("""
        <style>
            body {
                background-color: #EBECCC;
            }
            [data-testid="stAppViewContainer"] {
                background-color: #EBECCC;
            }

            h1 {
                color: #DCA278;
            }

            button[data-testid="baseButton-primary"] {
                background-color: #DCA278 !important;
                color: white !important;
                border: none !important;
                border-radius: 10px !important;
                padding: 0.6em 1.2em !important;
                font-size: 16px !important;
                font-weight: bold;
                transition: all 0.3s ease;
            }

            button[data-testid="baseButton-primary"]:hover {
                background-color: #C76D3A !important;
                transform: scale(1.02);
            }
            .stButton > button {
            background-color: #DCA278 !important;
            color: white !important;
        }


            .stDownloadButton button {
                background-color: #A6C9FF !important;
                color: black !important;
                border-radius: 10px !important;
                font-weight: 600;
                padding: 0.5em 1.2em !important;
            }

            .stDownloadButton button:hover {
                background-color: #8DBBFF !important;
            }
        </style>
    """, unsafe_allow_html=True)

    # --- Begin Centered Block ---
    st.markdown('<div class="main"><div class="page-border">', unsafe_allow_html=True)

    st.markdown("<div class='center-box'>", unsafe_allow_html=True)

    # Title
    st.markdown(
        """
        <h1 style='text-align: center; color: #DCA278;'>
            <span style='color:#DCA278;'>AI Digital Photobooth 
        </h1>
        <p style='text-align: center; font-size: 18px; color: #6b3e26; margin-top: -10px;'>
            Powered by Pix2Pix & your creativity ✨
        </p>
        """,
        unsafe_allow_html=True
    )

    # Image
    st.markdown("<div style='text-align: center;'>", unsafe_allow_html=True)
    st.image("/kaggle/input/photobooth-images/banner.jpg")
    st.markdown("</div>", unsafe_allow_html=True)

    # Welcome Text
    st.markdown(
        """
        <div style=' text-align:center;font-size: 18px; margin-top: 15px; margin-bottom: 25px;'>
            Welcome to the digital photobooth! Let's make some new memories.
            The app is powered by pix2pix which allows you to play with your picture and edit them.
        </div>
        """,
        unsafe_allow_html=True
    )

    # Centered Buttons
    col_spacer, col1, col2, col_spacer2 = st.columns([1, 2, 2, 1])
    with col1:
        if st.button("📷 Use Camera"):
            st.session_state.page = "camera"
    with col2:
        if st.button("🖼 Upload Photos"):
            st.session_state.page = "upload"

    # --- Close Centered Block ---
    st.markdown("</div>", unsafe_allow_html=True)
    st.stop()

# --- Frame Color & Captions ---


# --- Camera Input ---
if st.session_state.page == "camera":

    st.markdown("""
        <style>
            body {
                background-color: #EBECCC;
            }
            [data-testid="stAppViewContainer"] {
                background-color: #EBECCC;
            }

            h1 {
                color: #DCA278;
            }

            button[data-testid="baseButton-primary"] {
                background-color: #DCA278 !important;
                color: white !important;
                border: none !important;
                border-radius: 10px !important;
                padding: 0.6em 1.2em !important;
                font-size: 16px !important;
                font-weight: bold;
                transition: all 0.3s ease;
            }

            button[data-testid="baseButton-primary"]:hover {
                background-color: #C76D3A !important;
                transform: scale(1.02);
            }
            .stButton > button {
            background-color: #DCA278 !important;
            color: white !important;
        }


            .stDownloadButton button {
                background-color: #A6C9FF !important;
                color: black !important;
                border-radius: 10px !important;
                font-weight: 600;
                padding: 0.5em 1.2em !important;
            }

            .stDownloadButton button:hover {
                background-color: #8DBBFF !important;
            }
        </style>
    """, unsafe_allow_html=True)

    
    
    
    st.title("📷 Take Photos")
    img = st.camera_input("Take a photo")
    if img and len(st.session_state.photos) < 3:
        st.session_state.photos.append(Image.open(img))
    elif len(st.session_state.photos) >= 3:
        st.warning("You've already taken 3 photos!")

    if st.button("Clear All Photos"):
        st.session_state.photos = []

# --- Upload Input ---
elif st.session_state.page == "upload":
    
    st.markdown("""
        <style>
            body {
                background-color: #EBECCC;
            }
            [data-testid="stAppViewContainer"] {
                background-color: #EBECCC;
            }

            h1 {
                color: #DCA278;
            }

            button[data-testid="baseButton-primary"] {
                background-color: #DCA278 !important;
                color: white !important;
                border: none !important;
                border-radius: 10px !important;
                padding: 0.6em 1.2em !important;
                font-size: 16px !important;
                font-weight: bold;
                transition: all 0.3s ease;
            }

            button[data-testid="baseButton-primary"]:hover {
                background-color: #C76D3A !important;
                transform: scale(1.02);
            }
            .stButton > button {
            background-color: #DCA278 !important;
            color: white !important;
        }


            .stDownloadButton button {
                background-color: #A6C9FF !important;
                color: black !important;
                border-radius: 10px !important;
                font-weight: 600;
                padding: 0.5em 1.2em !important;
            }

            .stDownloadButton button:hover {
                background-color: #8DBBFF !important;
            }
        </style>
    """, unsafe_allow_html=True)
    

    st.title("🖼 Upload Photos")
    uploaded_files = st.file_uploader("Upload up to 3 images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)
    if uploaded_files:
        st.session_state.photos = [Image.open(f) for f in uploaded_files[:3]]
    
    if st.button("Clear All Photos"):
        st.session_state.photos = []

photos = st.session_state.photos
frame_color = st.color_picker("Choose Polaroid Frame Color 🎨", "#fdf6ec")
last_caption = st.text_input("Caption for Last Photo", "")

num_photos = len(st.session_state.photos)
captions = [""] * num_photos
if num_photos > 0:
    captions[-1] = last_caption


# --- Make Polaroid Strip ---
def make_polaroid_strip(images, captions, bg_color="#FFFFFF"):
    def hex_to_rgb(hex_color):
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2 ,4))
    num_photos = len(images)
    img_width, img_height = 320, 320  # Each photo size
    border = 20
    bottom_margin = 50
    spacing = 10

    # Total size of the final strip
    strip_width = img_width + 2 * border
    strip_height = num_photos * (img_height + spacing) + bottom_margin + border

    # Create base strip
    bg_rgb = hex_to_rgb(bg_color)
    strip = Image.new("RGBA", (strip_width, strip_height), (*bg_rgb, 255))

    font = ImageFont.truetype("DejaVuSans-Oblique.ttf", size=40) if os.path.exists("/usr/share/fonts/truetype/dejavu/DejaVuSans-Oblique.ttf") else None
    draw = ImageDraw.Draw(strip)

    y = border
    for idx, img in enumerate(images):
        resized = img.resize((img_width, img_height)).convert("RGBA")
        strip.paste(resized, (border, y))

        # Add caption if provided
        if captions[idx]:
            text = captions[idx]
            text_bbox = draw.textbbox((0, 0), text, font=font)
            text_width = text_bbox[2] - text_bbox[0]
            text_x = strip_width - border - text_width - 10  # right-align with padding
            text_y = y + img_height + 5
            draw.text((text_x, text_y), text, fill="#333", font=font,size=40)
           
        y += img_height + spacing

    return strip

# --- Display Polaroid ---
if photos:
    st.subheader("Polaroid Strip Preview")
    polaroid = make_polaroid_strip(photos, captions, frame_color)
    st.image(polaroid)

    buf = io.BytesIO()
    polaroid.save(buf, format="PNG")
    st.download_button("📥 Download Polaroid", data=buf.getvalue(), file_name="polaroid.png", mime="image/png")

# --- Pix2Pix Editing ---
st.subheader("🪄 Edit Photos with AI (Pix2Pix)")
edit_prompt = st.text_input("Describe the edit you want (e.g., 'make it vintage')")

if st.button("Apply Edit with Pix2Pix"):
    if not torch.cuda.is_available():
        st.error("⚠️ GPU not available! Please switch to GPU runtime.")
    elif not photos:
        st.warning("Upload or take a photo first.")
    else:
        with st.spinner("Loading model and applying edit..."):
            pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
                "timbrooks/instruct-pix2pix",
                torch_dtype=torch.float16,
                safety_checker=None,
                requires_safety_checker=False
            ).to("cuda")

            edited_images = []
            for img in photos:
                edited = pipe(prompt=edit_prompt, image=img, num_inference_steps=20, image_guidance_scale=1.5).images[0]
                edited_images.append(edited)

            st.success("✨ Edit applied!")
            edited_polaroid = make_polaroid_strip(edited_images, captions, frame_color)
            st.image(edited_polaroid, caption="Edited Polaroid Strip")

            buf = io.BytesIO()
            edited_polaroid.save(buf, format="PNG")
            st.download_button("📥 Download Edited Polaroid", data=buf.getvalue(), file_name="edited_polaroid.png", mime="image/png")   


Writing app.py


In [ ]:
!ngrok config add-authtoken 

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [5]:
from pyngrok import ngrok

import subprocess

# Kill any existing Streamlit processes
!pkill -f streamlit

# Start Streamlit using subprocess
process = subprocess.Popen(["streamlit", "run", "app.py"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)


public_url = ngrok.connect(8501,"http")
print("🚀 App is live at:", public_url)


🚀 App is live at: NgrokTunnel: "https://8fe5-34-132-238-11.ngrok-free.app" -> "http://localhost:8501"
